In [1]:
import modules.labelHelper as lh 
import modules.featureHelper as fh
from keras.callbacks import (ModelCheckpoint, LearningRateScheduler,
                             EarlyStopping, ReduceLROnPlateau,CSVLogger)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import warnings
import keras
from keras.callbacks import EarlyStopping
from keras.regularizers import l2 
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Dense, Dropout, Activation, Flatten,Conv1D, GlobalAveragePooling1D, MaxPooling1D,AveragePooling1D
from keras.layers import Conv2D, MaxPooling2D,Conv1D,GlobalMaxPooling1D,MaxPooling1D,average, concatenate,RepeatVector,Lambda,add,subtract,Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Input, Model
from sklearn import metrics as mt
from skimage.io import imshow
from sklearn import preprocessing
from matplotlib.patches import Rectangle
from keras import backend as K
from keras.layers.normalization import BatchNormalization
warnings.filterwarnings('ignore')
from modules.modelTools import *

Import an awesome Label helper
import an awesome feature helper


C:\Users\razer\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\razer\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\razer\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])

In [2]:
ll=lh.Label('..//2019_fall_labels.csv')
print("There are %d videos in total\n"%ll.video_count())
ll.get_trainee_info()

train_video_number,test_video_number=ll.train_test_split_on_people(0.2)
print(train_video_number)
print(test_video_number)

## set up feature parameters ##
folder = "C:\\2019_fall_video_features"
video_clips_length=30
time_lag=2
move_threshold=150
stride=video_clips_length


# train test split by people
train_video,train_video_label=fh.make_train_test_data_from_video_numbers(folder,train_video_number,video_clips_length,time_lag,move_threshold,stride,ll)
test_video,test_video_label=fh.make_train_test_data_from_video_numbers(folder,test_video_number,video_clips_length,time_lag,move_threshold,stride,ll)
print(train_video.shape)
print(train_video_label.shape)
print(test_video.shape)
print(test_video_label.shape)

There are 83 videos in total

There are 19 trainees in total

 Number of Videos from each person
Akhtar          -- [176 253 761] ----: 3
Cadeddu         -- [164 171] ----: 2
Crivelli        -- [225 276 333 425] ----: 4
Gahan           -- [257 345 378 384 486] ----: 5
Johnson         -- [441 615] ----: 2
Keith           -- [240 277 717] ----: 3
Kenigsberg      -- [ 78 440 455 471 881] ----: 5
Krabbe          -- [152 344 527 861] ----: 4
Marthur         -- [130 194 222 368 921] ----: 5
Mollengarden    -- [ 59 143 267 460] ----: 4
Moony           -- [294 301 361 498 539] ----: 5
Passoni         -- [207 237 895 942] ----: 4
Rozanski        -- [113 302 716] ----: 3
Satyanarayan    -- [ 16  74 236 358 436 457 503 537 557 578 599 632 689] ----: 13
Singla          -- [ 49 536 538] ----: 3
Sorokin         -- [ 91 226 507 530] ----: 4
Timburlake      -- [258 296 559 742] ----: 4
Tse             -- [ 11 283 414 427 562 928] ----: 6
Varun           -- [192 401 417 820] ----: 4
train on videos fro

In [3]:
l2_lambda=0.01
model=make_model(0.01,30,28)

In [4]:
# model.summary()

In [5]:
X_train=train_video/640-0.5
X_test=test_video/640-0.5
y_train=train_video_label/5
y_test=test_video_label/5
#     y_train=y_train
#     y_test=y_test
# print(X_train.shape,y_train.shape)
# print(X_test.shape,y_test.shape)

sample_of_trainningdata=X_train.shape[0]
sample_of_testdata=X_test.shape[0]

batch_size=64

train_gen=generator(X_train, y_train, batch_size,0.1)
val_gen=generator(X_test, y_test, batch_size,0)

#prepare for inout shape
clip_lenth=X_train.shape[1]
dimension=X_train.shape[2]

callbacks_list = get_callback_list_by_model('MTmodel')
model_h=model.fit_generator(train_gen,
                    steps_per_epoch=sample_of_trainningdata//batch_size,
                    epochs=30,
                    validation_data=val_gen,
                   validation_steps=sample_of_testdata//batch_size +1 ,
                    verbose=1,
                    callbacks=callbacks_list
 )

Epoch 1/30
26/26 [==============================] - 8s 294ms/step - loss: 0.7588 - DP_loss: 0.1055 - BD_loss: 0.0863 - E_loss: 0.1064 - FS_loss: 0.1134 - A_loss: 0.1844 - RC_loss: 0.0943 - val_loss: 0.7166 - val_DP_loss: 0.0854 - val_BD_loss: 0.0688 - val_E_loss: 0.1164 - val_FS_loss: 0.0988 - val_A_loss: 0.1471 - val_RC_loss: 0.1317

Epoch 00001: val_loss improved from inf to 0.71656, saving model to model_zoo/MTmodel.h5
Epoch 2/30
26/26 [==============================] - 1s 26ms/step - loss: 0.6084 - DP_loss: 0.0732 - BD_loss: 0.0741 - E_loss: 0.0853 - FS_loss: 0.0873 - A_loss: 0.1433 - RC_loss: 0.0771 - val_loss: 0.6450 - val_DP_loss: 0.0887 - val_BD_loss: 0.0757 - val_E_loss: 0.1257 - val_FS_loss: 0.0828 - val_A_loss: 0.0919 - val_RC_loss: 0.1122

Epoch 00002: val_loss improved from 0.71656 to 0.64503, saving model to model_zoo/MTmodel.h5
Epoch 3/30
26/26 [==============================] - 1s 26ms/step - loss: 0.5057 - DP_loss: 0.0578 - BD_loss: 0.0662 - E_loss: 0.0735 - FS_loss: 0

In [6]:
# callbacks_list = get_callback_list_by_model("mtmodel")
# #     train_proteins_shuffle, train_X_shuffle, train_y_shuffle = shuffle(train_proteins, train_X, train_y,random_state=randomS)


# model.fit(x=train_video, # create a list of inputs for embeddings
#         y=train_video_label, epochs=10, 
#         batch_size=64, verbose=1,

#         callbacks=callbacks_list         
#            )

In [8]:
# re=model.fit(X_test)

In [9]:
# len(re[0])

In [ ]:
# # train_gen=generator(train_video, train_video_label, 32,0.1)
# a,b=next(train_gen)